# COGS 108 - Final Project - Crime Watch Effectiveness and Distribution

# Overview

<span style="background-color: #FFFF00">Fill in later once have a bit more data analysis finished. Treat like an Abstract!</span>

# Names

- Alexander Schonken (A13331901)
- Jeffrey Chu        (A13613249)
- Jennifer Dich      (A14230996)
- Owen Clinton       (A13340650)

# Research Question

The question our group is trying to answer is: <b>What factors contribute to the types of crime committed in Los Angeles within a specific area throughout the day/night?</b> We also want to address what a data driven response would look like. One specific example we want to look into is whether street light coverage or a greater police presence contribute to lower overall crime rates in neighborhoods at night. We will use this data to consider whether moving police patrols to areas with less streetlight coverage would lead to a reduction in overall crime.

## Background and Prior Work

The datasets that we’re planning on using consist of streetlight data for the city of Los Angeles in combination with as much police data we can gather on locations where officers are called, the types of crimes committed, data on arrests made, and where officers stop people consistently. These datasets used together will help lead us to a more holistic view of the situation we are trying to analyze. Each of the datasets does not necessarily hold all of the data the others do in a homogenous way (i.e. some are missing years) but by increasing the amount of datasets we use and inferring connections based off of the data present, a clear picture will be painted and our call to action on what practices need to be put in place to increase the safety of Los Angeles will be backed up by valid data.

Before starting our exploratory data analysis of datasets we found on the subject, we don’t know a lot about the topic besides the inherent feeling of safety one gets when you’re in an area with street lights compared to a dark street. So our starting hypothesis is mainly based off of intuition and personal experience. The datasets we’ve found so far have a lot of information within them besides just location data, so we’ll have a very wide view of this narrow topic which we’ll be able to continually narrow our scope to reveal interesting answers and insightful conclusions.  

We weren’t able to find any other projects asking questions about the correlation between streetlights and police officer patrol locations. Our project seems to be fairly unique in the intersectionality of the data we’re using to reveal interesting insights into public safety. However, there have been projects created about streetlights and their affects on the number of crimes per year. An example of this is the report by Kinder Rice Institute for Urban Research that goes into how it was very difficult to recommend that cities increase streetlights to directly combat crime. Interestingly, on the other side of the argument is a research paper by CrimeLab New York in 2019 that found a significant drop in index crimes consisting of murder, aggravated assault, and other more "deadly" crimes. In addition to that finding, the paper also found that citizens living in the areas that received the lighting improvements were very pleased to have the additional infastructure in their neighborhoods. So the correlation between increasing street lighting and lowering crimes was very positively correlated in addition to the side effect of increasing the general happiness of citizens in the area.  

Since the research, that our group has been able to find, presents conclusions landing on both sides of the benefits of increasing street lighting to hep combat crime, our team's queston which more specifically focuses on police patrol positions relative to these well-lit areas should shed some new light on the subject and lead to increased insight on an apparently contested matter. Hopefully our project is able to contribute to the public good of Los Angeles (and by extension other cities) by discovering ways to increase the city's safety without increasing costs for taxpayers.

<b>References (include links):</b> 
<br>1) Kinder Rice Institute for Urban Researc - What Happens in the Shadows: Streetlights and How They Relate To Crime:
<br>https://kinder.rice.edu/sites/g/files/bxs1676/f/documents/Kinder%20Streetlights%20and%20Crime%20report.pdf
<br>2) CrimeLab New York - "Can Street Lighting Reduce Crime?":
<br>https://urbanlabs.uchicago.edu/projects/crime-lights-study

# Hypothesis


We hypothesize that areas with more street light coverage will have less overall crime than areas with less coverage. Therefore leading to a minimal need for police to monitor those areas, it will allow them the freedom to expand their range to cover areas that are less safe. 

# Datasets

Dataset Name: Crime Data from 2010 to Present
- Link to the dataset: https://data.lacity.org/api/views/63jg-8b9z/rows.csv?accessType=DOWNLOAD
- Number of observations:2.09 million
- Description: LA city's dataset of crime starting in 2010. It includes all of the crime reports made within LA city during that time.

Dataset Name: Arrest Data from 2010 to Present
- Link to the dataset: https://data.lacity.org/api/views/yru6-6re4/rows.csv?accessType=DOWNLOAD
- Number of observations: 1.32 million
- Description: LA city's dataset of arrests starting in 2010. It includes booking data for all arrests made within LA city during that time.

Dataset Name: Zillow Home Value Index
- Link to the dataset: http://files.zillowstatic.com/research/public/Zip/Zip_Zhvi_Summary_AllHomes.csv
- Number of observations: 15744
- Description: Zillow's home value index dataset sorted by ZIP code. It includes all of the median house values in a certain ZIP code.  

Dataset Name: Median List Price per Square Foot
- Link to the dataset: http://files.zillowstatic.com/research/public/Zip/Zip_MedianListingPricePerSqft_AllHomes.csv
- Number of observations: 15744
- Description: Zillow's dataset including the median list price per square foot for houses in different ZIP codes.

Dataset Name: Streetlight Locations
- Link to the dataset: https://data.lacity.org/api/geospatial/9ei6-svt8?method=export&format=GeoJSON
- Number of observations: 216738
- Description: Point locations of all the streelights managed by LA city.

Dataset Name: Los Angeles City Zip Codes
- Link to the dataset: https://opendata.arcgis.com/datasets/875d540d71e64b8696cc368865c2b640_0.geojson
- Number of observations: 157
- Description: Geo data for ZIP codes within the city of LA.

Dataset Name: LA County ZIP Code
- Link to the dataset: https://opendata.arcgis.com/datasets/70748ba37ecc418891e052e800437681_5.geojson
- Number of observations: 316
- Description: Geo data for ZIP codes within LA county.

Dataset Name: 2010 Census Populations by Zip Code
- Link to the dataset: https://data.lacity.org/api/views/nxs9-385f/rows.csv?accessType=DOWNLOAD
- Number of observations: 319
- Description: Dataset including population within ZIP codes of LA county.

We will use the crime and arrest datasets joined with the zip code datasets to analyze the amount of crime in each ZIP code of LA. We will tie this data to the streetlight locations to see if the amount of streetlights in a given zip code affect the amount of crime. The population and home value datasets are there so we can add controls to the crime data.

# Setup

<h1> To Do:</h1>

- Median population age verticle bar in vega graphs
- Change the house value to median house sold price

In [49]:
# Imports

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import folium.plugins as plugins
import shapely
import os

# Need to pip install
import altair as alt
import geopandas as gpd

# Configure libraries

# Setup seaborn
sns.set()
sns.set_context('talk')

# Change options for displaying DataFrames
pd.options.display.max_rows = 7
pd.options.display.max_columns = 50

# Round decimals in DataFrames
pd.set_option('precision', 2)

# Allow altair to display graphs in notebook
alt.renderers.enable('notebook')

#Setup altair to save graphs to external file
#def json_dir(data, data_dir = 'altairdata'):
    #os.makedirs(data_dir, exist_ok = True)
    #return alt.pipe(data, alt.to_json(filename = data_dir + '/{prefix}-{hash}.{extension}'))
#alt.data_transformers.register('json_dir', json_dir)
#alt.data_transformers.enable('json_dir', data_dir = 'mydata')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

File Paths for all the Datasets

In [2]:
LAArrestDataPath = r'ProjectData/LAArrestData.csv'
LACrimeDataPath = r'ProjectData/LACrimeData.csv'
LAStreetlightLocationPath = r'ProjectData/LAStreetlightLocations.geojson'
HomeValuePath = r'ProjectData/MedianPricePerSquareFoot.csv'
LAZipCodesPath = r'ProjectData/LAZipCodes.geojson'
LAZipCodePopulationPath = r'ProjectData/LAPopulationbyZipCode.csv'
LACityZipCodesPath = r'ProjectData/LACityZipCodes.geojson'

Initialize all datasets into geopandas and pandas dataframes

In [3]:
arrests = pd.read_csv(LAArrestDataPath)
crimes = pd.read_csv(LACrimeDataPath)
homeValue = pd.read_csv(HomeValuePath)
populations = pd.read_csv(LAZipCodePopulationPath)
streetlights = gpd.read_file(LAStreetlightLocationPath)
zipCodes = gpd.read_file(LAZipCodesPath)
cityzipCodes = gpd.read_file(LACityZipCodesPath)

# Data Cleaning

Drop columns and put crimes location into same format as arrests

In [4]:
arrests = arrests.drop(['Area ID','Area Name','Charge Group Code',
                        'Arrest Type Code','Charge','Address',
                        'Cross Street','Charge Description'], axis = 1)

crimes['Location'] = '(' + crimes['LAT'].map(str) + ', ' + crimes['LON'].map(str) + ')'

crimes = crimes.drop(['Date Rptd','AREA ','AREA NAME','Part 1-2','Crm Cd',
                      'Mocodes','Premis Cd','Premis Desc','Weapon Used Cd',
                      'Weapon Desc','Status','Crm Cd 1','Crm Cd 2','Crm Cd 3',
                      'Crm Cd 4','LOCATION','Cross Street','LON','LAT'], axis = 1)

arrests.columns = ['Report ID','Date','Time','Reporting District', 
                  'Arrest Age','Arrest Sex','Arrest Descent',
                   'Arrest Description','Location']
crimes.columns = ['Report ID','Date','Time','Reporting District',
                  'Crime Description','Victim Age','Victim Sex',
                  'Victim Descent','Status','Location']

Drop Nan data

In [5]:
arrests = arrests.dropna(subset = ['Time', 'Arrest Age', 'Arrest Sex', 'Arrest Descent', 'Arrest Description'])
crimes = crimes.dropna(subset = ['Time', 'Crime Description', 'Victim Age', 'Victim Sex', 'Victim Descent'])

Convert string location points to geometry for geopandas

In [6]:
def string_to_point(string):
    string_tuple = tuple(map(float, string.replace('(','').replace(')','').split(',')))
    point = shapely.geometry.Point(string_tuple[1],string_tuple[0])
    return point

In [7]:
arrestsgeometry = arrests['Location'].apply(string_to_point)
crimesgeometry = crimes['Location'].apply(string_to_point)

Add Zip Codes to crimes and arrest and streetlight dataframe

In [8]:
arrests_geo =  gpd.GeoDataFrame(arrests['Report ID'], geometry = arrestsgeometry)
crimes_geo = gpd.GeoDataFrame(crimes['Report ID'], geometry = crimesgeometry)

arrests_geo.crs = zipCodes.crs
crimes_geo.crs = zipCodes.crs

In [9]:
arrestswithzip = gpd.sjoin(arrests_geo,zipCodes, how = 'inner', op = 'within')
crimeswithzip = gpd.sjoin(crimes_geo, zipCodes, how = 'inner', op = 'within')
streetlights = gpd.sjoin(streetlights, zipCodes, how = 'inner', op = 'within')

In [10]:
arrestsmergeframe = pd.DataFrame(arrestswithzip)
crimesmergeframe = pd.DataFrame(crimeswithzip)

In [11]:
arrests = pd.merge(arrests, arrestsmergeframe, on = 'Report ID')
crimes = pd.merge(crimes, crimesmergeframe, on = 'Report ID')

In [12]:
arrests = arrests.drop(['OBJECTID','index_right'], axis = 1)
crimes = crimes.drop(['OBJECTID','index_right'], axis = 1)

In [13]:
streetlightsdf = pd.DataFrame(streetlights)

Change time to actual time variables

In [14]:
def gethour(integer):
    return int(integer/100)

In [15]:
arrests['Time'] = arrests['Time'].astype(int)

In [16]:
crimes['Hour'] = crimes['Time'].apply(gethour)
arrests['Hour'] = arrests['Time'].apply(gethour)

Make a list of unique zipcodes

In [17]:
crimesarray = crimes['ZIPCODE'].unique()
arrestsarray = arrests['ZIPCODE'].unique()

In [18]:
zipCodeArray = list(set(crimesarray) | set(arrestsarray))

In [19]:
zipCodedf = pd.DataFrame(zipCodeArray, columns = ['Zip'])

Only include housing, population data where stuff happens

In [20]:
homeValue['Zip'] = homeValue['RegionName'].astype(str)
populations['Zip'] = populations['Zip Code'].astype(str)

In [21]:
homeValue = pd.merge(zipCodedf, homeValue, how = 'left', on = 'Zip')
populations = pd.merge(zipCodedf, populations, how = 'left', on = 'Zip')

In [22]:
homeValue = homeValue.fillna(0)
populations = populations.fillna(0)

Add area to zip code geodataframes

In [23]:
cityzipCodes['Area'] = cityzipCodes['geometry'].to_crs({'init': 'epsg:3395'}).map(lambda p: p.area / 10**6)

Change one letter description strings in crimes and arrests dataframes to what they represent

In [24]:
def sexToString(string):
    if string == 'F':
        return 'Female'
    elif string == 'M':
        return 'Male'
    else:
        return 'Unknown'
descentDict = {
    'A': 'Other Asian',
    'B': 'Black',
    'C': 'Chinese',
    'D': 'Cambodian',
    'F': 'Filipino',
    'G': 'Guamanian',
    'H': 'Hispanic/Latin/Mexican',
    'I': 'American Indian/Alaskan Native',
    'J': 'Japanese',
    'K': 'Korean',
    'L': 'Laotian',
    'O': 'Other',
    'P': 'Pacific Islander',
    'S': 'Samoan',
    'U': 'Hawaiian',
    'V': 'Vietnamese',
    'W': 'White',
    'X': 'Unknown',
    'Z': 'Asian Indian',
    '-': 'Unknown'
}
def descentToString(string):
    return descentDict[string]

In [25]:
arrests['Arrest Sex'] = arrests['Arrest Sex'].apply(sexToString)
crimes['Victim Sex'] = crimes['Victim Sex'].apply(sexToString)
arrests['Arrest Descent'] = arrests['Arrest Descent'].apply(descentToString)
crimes['Victim Descent'] = crimes['Victim Descent'].apply(descentToString)

# Data Analysis & Results

- 3 maps 
- 1 choropleth with counts popup including total, population, #streetlights, house value
- 1 heatmap with time
- 1 choropleth with popup graphs

- statistical analysis of streetlights

<b>We will start off with a couple mapped visualizations of our data</b>

To do this we first tally up the total amount of crimes and arrests committed and made in each zip code

In [26]:
crimeCountDict = crimes['ZIPCODE'].value_counts()
arrestCountDict = arrests['ZIPCODE'].value_counts()
streetlightsDict = streetlightsdf['ZIPCODE'].value_counts()

In [27]:
crimeCounts = pd.DataFrame.from_dict(crimeCountDict).reset_index()
arrestCounts = pd.DataFrame.from_dict(arrestCountDict).reset_index()
streetlightsCounts = pd.DataFrame.from_dict(streetlightsDict).reset_index()

In [28]:
crimeCounts.columns = ['Zip','Crime Count']
arrestCounts.columns = ['Zip', 'Arrest Count']
streetlightsCounts.columns = ['Zip', 'Streetlight Count']

In [29]:
counts = pd.merge(crimeCounts, arrestCounts, how = 'outer', on = 'Zip')
counts = pd.merge(counts, streetlightsCounts, how = 'left', on = 'Zip')

In [30]:
counts = counts.fillna(0)

In [31]:
counts['Total'] = counts['Crime Count'] + counts['Arrest Count']

We then map out each zip code and add pop ups to each one to display data for each zip code

In [32]:
choroplethPopup = folium.Map([34.055862, -118.326904], max_zoom = 12)
zipcodeLayer = folium.FeatureGroup(name = 'Zip Codes')
transparent = {'fillColor': '#00000000', 'color': '#00000000'}
folium.Choropleth(
    geo_data = cityzipCodes,
    name = 'Choropleth',
    data = counts,
    columns = ['Zip', 'Total'],
    key_on = 'feature.properties.ZIPCODE',
    fill_color = 'BuPu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Total Crimes').add_to(choroplethPopup)

for i in zipCodeArray:
    datarow = counts.where(counts['Zip'] == i).dropna().iloc[0]
    gs = folium.GeoJson(zipCodes.loc[zipCodes['ZIPCODE'] == i], style_function = lambda x: transparent)
    zipCodelabel = i
    crimeslabel = datarow['Crime Count']
    arrestslabel = datarow['Arrest Count']
    totallabel = datarow['Total']
    populationlabel = populations.where(populations['Zip'] == i).dropna().iloc[0]['Total Population']
    homeValuelabel = int(homeValue.where(homeValue['Zip'] == i).dropna().iloc[0]['2019-10'])
    streetlightslabel = datarow['Streetlight Count']
    popuphtml = """
            <html>
                <body>
                    <h1>%s</h1>
                    <p>Number of Crimes Commited: %s</p>
                    <p>Number of Arrests Made: %s</p>
                    <p>Total: %s</p>
                    <p>Median Home Price Per Square Foot: %s</p>
                    <p>Population: %s</p>
                    <p>Number of Streetlights: %s</p>
                </body>
            </html>"""%(zipCodelabel, str(crimeslabel), str(arrestslabel), str(totallabel), 
                        str(homeValuelabel), str(populationlabel), str(streetlightslabel))
    folium.Popup(popuphtml,max_width = 250, sticky = True).add_to(gs)
    gs.add_to(zipcodeLayer)
choroplethPopup.add_child(zipcodeLayer)
folium.LayerControl().add_to(choroplethPopup)

In [33]:
choroplethPopup.save('LabeledChoropleth.html')

Next we move on to a dynamic heatmap of crime activity

In [34]:
crimesHeatMap = crimes[['Report ID', 'Location', 'Hour']]
arrestsHeatMap = arrests[['Report ID', 'Location', 'Hour']]

In [35]:
heatMap  = pd.merge(crimesHeatMap, arrestsHeatMap, how = 'outer', on = ['Report ID', 'Location', 'Hour'])

In [36]:
heatMap['Hour'] = heatMap['Hour'].replace(24.0, 0.0)

In [37]:
heatMap['Lat'], heatMap['Lon'] = heatMap['Location'].str.split(',').str

In [38]:
heatMap['Lat'] = heatMap['Lat'].str.replace('(','').astype(float)
heatMap['Lon'] = heatMap['Lon'].str.replace(')','').astype(float)

In [39]:
heatMap['count'] = 1

In [40]:
hoursofday = ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', 
              '06:00', '07:00', '08:00', '09:00', '10:00', '11:00',
              '12:00', '13:00', '14:00', '15:00', '16:00', '17:00',
              '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']

In [41]:
hourList = []
for i in heatMap.Hour.sort_values().unique():
    hourList.append(heatMap.loc[heatMap.Hour == i, ['Lat', 'Lon', 'count']].groupby(['Lat', 'Lon']).sum().reset_index().values.tolist())

In [42]:
timeHeatMap = folium.Map([34.055862, -118.326904], max_zoom = 12)
plugins.HeatMapWithTime(
    hourList,
    index = hoursofday,
    radius = 5,
    gradient = {0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'},
    min_opacity = 0.5,
    max_opacity = 0.8,
    use_local_extrema = True
    ).add_to(timeHeatMap)

In [43]:
timeHeatMap.save('HeatMap.html')

Now we make charts of our data and put them on the map

In [44]:
crimesChartFrame = crimes[['Victim Age', 'Victim Sex', 'Victim Descent', 'Crime Description', 'ZIPCODE']]
arrestsChartFrame = arrests[['Arrest Age', 'Arrest Sex', 'Arrest Descent', 'Arrest Description', 'ZIPCODE']]

In [83]:
interestingzipCodes = [90003, 90028, 90210, 90291, 90042, 90005, 90012]

In [84]:
ageSexDict = {}

for i in interestingzipCodes:
    crime_frame = crimesChartFrame.where(crimesChartFrame['ZIPCODE'] == str(i)).dropna()
    arrest_frame = arrestsChartFrame.where(arrestsChartFrame['ZIPCODE'] == str(i)).dropna()
    population_frame = populations.where(populations['Zip'] == str(i)).dropna()
    
    rule = alt.Chart(population_frame).mark_rule(color = 'red').encode(
        x = 'Median Age:O'
    ).properties(width = 300)

    crimesAgeSexChart = alt.Chart(crime_frame).mark_bar().encode(
        x = 'Victim Age:O',
        y = 'count(Victim Age):Q', color = 'Victim Sex'
    ).transform_filter(
        alt.FieldGTPredicate(field = 'Victim Age', gt = 0)
    ).properties(height = 200,width = 300)
    
    crimesSexes = alt.Chart(crime_frame).mark_bar().encode(
        x = 'count()',
        y = 'Victim Sex:O',
        color = 'Victim Sex'
    ).properties(height = 200, width = 100)
    
    arrestsAgeSexChart = alt.Chart(arrest_frame).mark_bar().encode(
        x = 'Arrest Age:O',
        y = 'count(Arrest Age):Q', color = 'Arrest Sex'
    ).transform_filter(
        alt.FieldGTPredicate(field = 'Arrest Age', gt = 0)
    ).properties(height = 200, width = 300)
    
    arrestsSexes = alt.Chart(arrest_frame).mark_bar().encode(
        x = 'count()',
        y = 'Arrest Sex:O',
        color = 'Arrest Sex'
    ).properties(height = 200, width = 100)
    
    x = alt.layer(crimesAgeSexChart, rule)|crimesSexes
    y = alt.layer(arrestsAgeSexChart, rule)|arrestsSexes
    ageSexDict[i] = x&y

In [85]:
ageDescentDict = {}
for i in interestingzipCodes:
    crime_frame = crimesChartFrame.where(crimesChartFrame['ZIPCODE'] == str(i)).dropna()
    arrest_frame = arrestsChartFrame.where(arrestsChartFrame['ZIPCODE'] == str(i)).dropna()
    population_frame = populations.where(populations['Zip'] == str(i)).dropna()
    
    rule = alt.Chart(population_frame).mark_rule(color = 'red').encode(
        x = 'Median Age:O'
    ).properties(width = 300)
    
    crimesAgeDescentChart = alt.Chart(crime_frame).mark_bar().encode(
        x = 'Victim Age:O',
        y = 'count(Victim Age):Q', color = 'Victim Descent'
    ).transform_filter(
        alt.FieldGTPredicate(field = 'Victim Age', gt = 0)
    ).properties(height = 200, width = 300)
    
    crimesDescents = alt.Chart(crime_frame).mark_bar().encode(
        x = 'count()',
        y = 'Victim Descent:O',
        color = 'Victim Descent'
    ).properties(height = 200, width = 100)
    
    arrestsAgeDescentChart = alt.Chart(arrest_frame).mark_bar().encode(
        x = 'Arrest Age:O',
        y = 'count(Arrest Age):Q', color = 'Arrest Descent'
    ).transform_filter(
        alt.FieldGTPredicate(field = 'Arrest Age', gt = 0)
    ).properties(height = 200, width = 300)
    
    arrestsDescents = alt.Chart(arrest_frame).mark_bar().encode(
        x = 'count()',
        y = 'Arrest Descent:O',
        color = 'Arrest Descent'
    ).properties(height = 200, width = 100)
    
    x = alt.layer(crimesAgeDescentChart, rule)|crimesDescents
    y = alt.layer(arrestsAgeDescentChart, rule)|arrestsDescents
    ageDescentDict[i] = x&y

In [86]:
descriptionDict = {}
for i in interestingzipCodes:
    crime_frame = crimesChartFrame.where(crimesChartFrame['ZIPCODE'] == str(i)).dropna()
    arrest_frame = arrestsChartFrame.where(arrestsChartFrame['ZIPCODE'] == str(i)).dropna()
    
    crimeDescChart = alt.Chart(crime_frame).transform_aggregate(
        count = 'count()',
        groupby = ['Crime Description']
    ).transform_window(
        rank = 'rank(count)',
        sort = [alt.SortField('count', order = 'descending')]
    ).mark_bar().encode(
        x = alt.X('Crime Description:O',
                 sort = alt.EncodingSortField(field = 'count',
                                             op = 'sum',
                                             order = 'descending')),
        y = alt.Y('count:Q')
    ).transform_filter(
        alt.datum.rank < 5
    ).properties(height = 200, width = 300)
    
    arrestDescChart = alt.Chart(arrest_frame).transform_aggregate(
        count = 'count()',
        groupby = ['Arrest Description']
    ).transform_window(
        rank = 'rank(count)',
        sort = [alt.SortField('count', order = 'descending')]
    ).mark_bar().encode(
        x = alt.X('Arrest Description:O',
                 sort = alt.EncodingSortField(field = 'count',
                                             op = 'sum',
                                             order = 'descending')),
        y = alt.Y('count:Q')
    ).transform_filter(
        alt.datum.rank < 5
    ).properties(height = 200, width = 300)
    
    descriptionDict[i] = crimeDescChart | arrestDescChart

In [87]:
choroplethGraph = folium.Map([34.055862, -118.326904], max_zoom = 12)
ageSexLayer = folium.FeatureGroup(name = 'Age vs Sex', show = False)
ageDescentLayer = folium.FeatureGroup(name = 'Age vs Descent', show = False)
descriptionLayer = folium.FeatureGroup(name = 'Descriptions')
transparent = {'fillColor': '#00000000', 'color': '#00000000'}
folium.Choropleth(
    geo_data = cityzipCodes,
    name = 'Choropleth',
    data = counts,
    columns = ['Zip', 'Total'],
    key_on = 'feature.properties.ZIPCODE',
    fill_color = 'BuPu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Total Crimes').add_to(choroplethGraph)

for i in interestingzipCodes:
    gsAS = folium.GeoJson(cityzipCodes.loc[cityzipCodes['ZIP'] == i])
    gsAD = folium.GeoJson(cityzipCodes.loc[cityzipCodes['ZIP'] == i])
    gsD = folium.GeoJson(cityzipCodes.loc[cityzipCodes['ZIP'] == i])
    zipCodelabel = str(i)
    folium.Popup(max_width = 750, sticky = True).add_child(
        folium.VegaLite(ageSexDict[i], width = 750, height = 400)).add_to(gsAS)
    folium.Popup(max_width = 750, sticky = True).add_child(
        folium.VegaLite(ageDescentDict[i], width = 750, height = 400)).add_to(gsAD)
    folium.Popup(max_width = 750, sticky = True).add_child(
        folium.VegaLite(descriptionDict[i], width = 700, height = 200)).add_to(gsD)
    ageSexLayer.add_child(gsAS)
    ageDescentLayer.add_child(gsAD)
    descriptionLayer.add_child(gsD)
    
    
choroplethGraph.add_child(ageSexLayer)
choroplethGraph.add_child(ageDescentLayer)
choroplethGraph.add_child(descriptionLayer)
folium.LayerControl().add_to(choroplethGraph)

In [88]:
choroplethGraph.save('GraphsChoropleth.html')

# Ethics & Privacy

Our objective is not to identify and analyze crimes, but only to retrieve data of the location of the crimes. We are observing whether the rate of crimes are reduced in relation to the presence of streetlights, resulting in a lower amount of police patrol in the area. Because the data in question was released publicly, a majority of private information regarding the identity of the perpetrator of the crime was removed. The police stop data only includes the arrested’s age, sex, and descent as personal identifiers, none of which breach the standards set by the Safe Harbour Method of data anonymization. Although unique identifiers have been removed for privacy concerns, the data that remains in our analyses may possibly be affected by racial and socioeconomic biases in relation to the area of the crimes committed. It is important to consider the idea that the tendency of criminals in one area to be of a certain ethnicity in our data is intrinsically linked to socioeconomic factors of the location in question. 

# Conclusion & Discussion

*Fill in your discussion information here*